In [1]:
# Programming evacuation systems using Crowd Simulation
# Agent-Based Modelling
# Loading the pygame package
import pygame
# Importing locals
from pygame.locals import *
# Other packages
import sys
import numpy as np
import numpy.random as random
import math
import time
import import_ipynb
from additional_functions import *
from dataset_cm_cv_cr import positionmatrix,nr_experiments,nr_agents


#Panic Force
def apply_panic_force_percentage(agents, panic_multiplier=3, percentage=25):
    """Apply a panic force to a percentage of agents by increasing their desired speed."""
    num_agents_to_affect = int(len(agents) * (percentage / 100))
    selected_agents = np.random.choice(agents, num_agents_to_affect, replace=False)
    for agent in selected_agents:
        agent.dSpeed *= panic_multiplier
        
#Herding Force (not very noticeable)
#def apply_herding_force(agents, herding_radius=50, herding_strength=2):
#    for agent in agents:
#        neighbors = [other for other in agents if np.linalg.norm(agent.pos - other.pos) < herding_radius and other != agent]
#        
#        if neighbors:
#            # Compute the average destination of neighbors
#            avg_dest = np.mean([neighbor.pos for neighbor in neighbors], axis=0)
#            
#            # Adjust the agent's direction towards this average destination
#            new_direction = normalize(avg_dest - agent.pos)
#            agent.direction = new_direction * herding_strength
#            # Optionally adjust velocity as well
#            agent.aVelocity = normalize(agent.aVelocity + new_direction * herding_strength)
#            
#    apply_herding_force(agents)

#data_matrix = np.loadtxt("data/room1_twodoors.txt", dtype=float)
agent_logs = [] #Initialize Array
data_matrix = np.zeros((nr_experiments*nr_agents, 4)) # Delete after first run

# Making sure we can run experiments one by one
j = 0 # add on after running

# Checking if all are executed
if j == nr_agents:
    print("nr of experiments reached")
    sys.exit()

# Initializing Pygame and font
pygame.init()
pygame.font.init()
timefont = pygame.font.SysFont('Comic Sans MS', 30)

"""

Creating a screen with a room that is smaller than then screen

"""


# Size of the screen
width = 800
height = 800
size = width, height # Do not adjust this

# Creating screen
roomscreen = pygame.display.set_mode(size, pygame.RESIZABLE)

# Making background white and creating colors
WHITE = (255,255,255)
RED = (255,0,0)
GREEN = (0,255,0)
BLACK = (0,0,0)
background_color = WHITE
roomscreen.fill(background_color)
pygame.display.update()

# Defining clock
clock = pygame.time.Clock()

# Move world view
world_offset = [0,0]

# Creating evacuation object class
class Agent(object):
    def __init__(self):

        self.mass = random.uniform(40,90)
        self.radius = 30 #random.uniform(15,30)
        # random initialize a agent

        self.x = np.random.uniform(100 + self.radius, 1284 - self.radius)
        self.y = np.random.uniform(50 + self.radius, 784 - self.radius)
        self.pos = np.array([self.x, self.y])
        
        self.aVelocityX = 0 #random.uniform(0,1.6)
        self.aVelocityY = 0 #random.uniform(0,1.6)
        self.aVelocity = np.array([self.aVelocityX, self.aVelocityY])
        #self.actualV = np.array([0.0, 0.0])
        
        self.dest = np.array([642,784])

        #self.dest = np.array([random.uniform(100,1184), random.uniform(50,734)])

        self.direction = normalize(self.dest - self.pos)
        #self.direction = np.array([0.0, 0.0])

        self.dSpeed = 12 #random.uniform(0.3,2.3) #1.8
        self.dVelocity = self.dSpeed*self.direction #dV = dS x direction

        self.acclTime = 0.5 #random.uniform(8,16) #10.0
        self.drivenAcc = (self.dVelocity - self.aVelocity)/self.acclTime #desired vs actual velocity

        self.bodyFactor = 120000
        self.F = 2000
        self.delta = 0.08*50 #random.uniform(0.8,1.6) #0.8 #0.08

        self.Goal = 0
        self.time = 0.0
        self.countcollision = 0

        print('X and Y Position:', self.pos)
        print('self.direction:', self.direction)

    def velocity_force(self): # function to adapt velocity
        deltaV = self.dVelocity - self.aVelocity #change in velocity
        if np.allclose(deltaV, np.zeros(2)):
            deltaV = np.zeros(2)
        return deltaV*self.mass/self.acclTime


    def f_ij(self, other): # interaction with people
        r_ij = self.radius + other.radius
        d_ij = np.linalg.norm(self.pos - other.pos)
        e_ij = (self.pos - other.pos)/d_ij
        value = self.F*np.exp((r_ij-d_ij)/(self.delta))*e_ij
        + self.bodyFactor*g(r_ij-d_ij)*e_ij

        if d_ij <= r_ij:
            self.countcollision += 1

        return value

    def f_ik_wall(self, wall): # interaction with the wall in the room
        r_i = self.radius
        d_iw,e_iw = distance_agent_to_wall(self.pos,wall)
        value = -self.F*np.exp((r_i-d_iw)/self.delta)*e_iw # Assume wall and people give same force
        + self.bodyFactor*g(r_i-d_iw)*e_iw
        return value

    def update_dest(self):
        if self.pos[0] < 642:
            self.dest = np.array([371,784])
          #  door1_left_edge = np.array([281, 784])
          #  door1_right_edge = np.array([461, 784])
          #  min_distance = np.inf
          #  closest_point = None
            
          #  for i in range(door1_left_edge[0], door1_right_edge[0] + 1):
          #      d = np.sqrt((self.pos[0] - i)**2 + (self.pos[1] - 0)**2)
          #      if d < min_distance:
          #          min_distance = d
          #          closest_point = np.array([i, 784])
          #  
          #  if self.pos[0] <= 371:
          #      if np.abs(np.linalg.norm(door1_left_edge - closest_point)) < 2*self.radius:
          #          while np.abs(np.linalg.norm(door1_left_edge - closest_point)) < 2*self.radius:
          #              self.dest = closest_point + self.radius
          #              break
          #      else:
          #          self.dest = closest_point
          #  else:
          #      if np.abs(np.linalg.norm(door1_left_edge - closest_point)) < 2*self.radius:
          #          while np.abs(np.linalg.norm(door1_left_edge - closest_point)) < 2*self.radius:
          #              self.dest = closest_point - self.radius
          #              break
          #      else:
          #          self.dest = closest_point


        elif self.pos[0] >= 642:
            self.dest = np.array([1013,784])
         #   door2_left_edge = np.array([923, 784])
         #   door2_right_edge = np.array([1103, 784])
         #   min_distance = np.inf
         #   closest_point = None
         #   for i in range(door2_left_edge[0], door2_right_edge[0] + 1):
         #       d = np.sqrt((self.pos[0] - i)**2 + (self.pos[1] - 0)**2)
         #       if d < min_distance:
         #           min_distance = d
         #           closest_point = np.array([i, 784])
         #   
         #   if self.pos[0] <= 1013:
         #       if np.abs(np.linalg.norm(door2_left_edge - closest_point)) < 2*self.radius:
         #           while np.abs(np.linalg.norm(door2_left_edge - closest_point)) < 2*self.radius:
         #               self.dest = closest_point + self.radius
         #               break
         #       else:
         #           self.dest = closest_point
         #   else:
         #       if np.abs(np.linalg.norm(door2_left_edge - closest_point)) < 2*self.radius:
         #           while np.abs(np.linalg.norm(door2_left_edge - closest_point)) < 2*self.radius:
         #               self.dest = closest_point - self.radius
         #               break
         #       else:
         #           self.dest = closest_point
            
            
def main():
    # Now to let multiple objects move to the door we define
    agent_color = GREEN
    line_color = BLACK

    # Making room
    room_height = 734 # height of the room
    room_width = 1184 # width of the room
    room_left = 100 # left pixels coordinate
    room_top = 50 # top pixels coordeinate

    """

    Now we need to create the doors through which objects will leave in case of evacuation
    This door's position can be determined using:

    """
    # Door 1
    door1_xright = 461 
    door1_xleft = 281

    # Door 2
    door2_yleft = 923
    door2_yright = 1103

    walls = [[room_left, room_top, room_left + room_width, room_top], #top wall
             [room_left, room_top, room_left, room_top + room_height], #left wall
             [room_left + room_width, room_top, room_left + room_width, room_top + room_height], #right wall
             [room_left, room_top + room_height, door1_xleft, room_top + room_height],
             [door1_xright, room_top + room_height, door2_yleft, room_top + room_height],
             [door2_yright, room_top + room_height, room_left + room_width, room_top + room_height]]


    # initialize agents
    agents = []

    def positions(agents):
        for i in range(nr_agents):
            agent = Agent()
            agent.x = positionmatrix[j*nr_agents+i][0]
            agent.y = positionmatrix[j*nr_agents+i][1]
            agent.pos = np.array([agent.x, agent.y])
            agent.radius = 30 #random.uniform(15,30)
            agent.mass = positionmatrix[j*nr_agents+i][3]
            agent.dSpeed = positionmatrix[j*nr_agents+i][4]
            #agent.pos = np.array([round(700 + 280*math.cos((i* 1/(nr_agents + 1) + 0.5)*math.pi) - 50),
            #round(400 + 280*math.sin((i * 1/(nr_agents+1) + 0.5)*math.pi))])
            agents.append(agent)
        apply_panic_force_percentage(agents, percentage=50) #apply panic force

    positions(agents)

    run = True
    count = 0
    start_time = time.time()
    while run:

        # Updating time
        if count < nr_agents-2:
            current_time = time.time()
            elapsed_time = current_time - start_time
        else:
            for agent_i in agents:
                data_matrix[(j+1)*nr_agents - 2][0] = elapsed_time
                data_matrix[(j+1)*nr_agents - 1][0] = elapsed_time
                agents.remove(agent_i)
            for k in range(j*nr_agents, (j+1)*nr_agents):
                data_matrix[k][1] = elapsed_time

        # Finding delta t for this frame
        dt = clock.tick(70)/1000

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                (x, y) = pygame.mouse.get_pos()
                print(x, y)

        global world_offset

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            world_offset[0] = world_offset[0] + 20
        if keys[pygame.K_RIGHT]:
            world_offset[0] = world_offset[0] - 20
        if keys[pygame.K_UP]:
            world_offset[1] = world_offset[1] + 20
        if keys[pygame.K_DOWN]:
            world_offset[1] = world_offset[1] - 20

        roomscreen.fill(background_color)

        # draw walls
        for wall in walls:
            start_posw = np.array([wall[0],wall[1]])
            end_posw = np.array([wall[2],wall[3]])
            start_posx = start_posw
            end_posx = end_posw
            pygame.draw.line(roomscreen, line_color, start_posx + world_offset, end_posx + world_offset, 3)

        for agent_i in agents:
            agent_i.update_dest()
            agent_i.direction = normalize(agent_i.dest - agent_i.pos)
            agent_i.dVelocity = agent_i.dSpeed*agent_i.direction
            aVelocity_force = agent_i.velocity_force()
            people_interaction = 0.0
            wall_interaction = 0.0

            for agent_j in agents:
                if agent_i == agent_j: continue
                people_interaction += agent_i.f_ij(agent_j)

            for wall in walls:
                wall_interaction += agent_i.f_ik_wall(wall)

            sumForce = aVelocity_force + people_interaction + wall_interaction + random.random()
            dv_dt = sumForce/agent_i.mass
            agent_i.aVelocity = agent_i.aVelocity + dv_dt*dt
            agent_i.pos = agent_i.pos + agent_i.aVelocity*dt


            if agent_i.pos[0] > 1334 or agent_i.pos[0] < 50 or agent_i.pos[1] > 809 or agent_i.pos[1] < 25:
                main()
                sys.exit()

        for agent_i in agents:

            agent_i.time += clock.get_time() / 1000
            data_matrix[count+j*nr_agents][0] =  agent_i.time
            start_position = [0, 0]
            start_position[0] = int(agent_i.pos[0]) + world_offset[0]
            start_position[1] = int(agent_i.pos[1]) + world_offset[1]

            end_position = [0, 0]
            end_position[0] = int(agent_i.pos[0] + agent_i.aVelocity[0]) + world_offset[0]
            end_position[1] = int(agent_i.pos[1] + agent_i.aVelocity[1]) + world_offset[1]

            end_positionDV = [0, 0]
            end_positionDV[0] = int(agent_i.pos[0] + agent_i.dVelocity[0]) + world_offset[0]
            end_positionDV[1] = int(agent_i.pos[1] + agent_i.dVelocity[1]) + world_offset[1]

            if start_position[1] >= 784 + world_offset[1] and agent_i.Goal == 0:
                exit_time = agent_i.time  # Assuming agent_i.time records the elapsed time for the agent
                agent_logs.append([agent_i.radius, exit_time])
                agent_i.Goal = 1
                # print('Time to Reach the Goal:', agent_i.time)
            
            # Save agent logs to a txt file
            agent_array = np.array(agent_logs)
            #np.savetxt('agent_data.txt', agent_array, fmt='%f', header='Radius Exit_Time', comments='')


            if start_position[1] > 784 + world_offset[1] or start_position[1] < 51 + world_offset[1]:
                data_matrix[count+j*nr_agents][2] = count
                data_matrix[count+j*nr_agents][3] = agent_i.countcollision
                count += 1
                agents.remove(agent_i)

            pygame.draw.circle(roomscreen, agent_color, start_position, round(agent_i.radius), 3)
            pygame.draw.line(roomscreen, agent_color, start_position, end_positionDV, 2)
        pygame.draw.line(roomscreen, [255,60,0], start_position, end_positionDV, 2)

        # Present text on screen
        timestr = "Time: " +  str(elapsed_time)
        timesurface = timefont.render(timestr, False, (0, 0, 0))
        roomscreen.blit(timesurface,(0,0))
        # Update the screen
        pygame.display.flip()


    pygame.quit()
main()
np.savetxt('seca302_twodoors_r30', data_matrix)



pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
importing Jupyter notebook from additional_functions.ipynb
importing Jupyter notebook from dataset_cm_cv_cr.ipynb
[(0.0, 0.0), (0.0, 156.8), (0.0, 313.6), (0.0, 470.40000000000003), (0.0, 627.2), (128.4, 0.0), (128.4, 156.8), (128.4, 313.6), (128.4, 470.40000000000003), (128.4, 627.2), (256.8, 0.0), (256.8, 156.8), (256.8, 313.6), (256.8, 470.40000000000003), (256.8, 627.2), (385.20000000000005, 0.0), (385.20000000000005, 156.8), (385.20000000000005, 313.6), (385.20000000000005, 470.40000000000003), (385.20000000000005, 627.2), (513.6, 0.0), (513.6, 156.8), (513.6, 313.6), (513.6, 470.40000000000003), (513.6, 627.2), (642.0, 0.0), (642.0, 156.8), (642.0, 313.6), (642.0, 470.40000000000003), (642.0, 627.2), (770.4000000000001, 0.0), (770.4000000000001, 156.8), (770.4000000000001, 313.6), (770.4000000000001, 470.40000000000003), (770.4000000000001, 627.2), (898.8000000000001, 

KeyboardInterrupt: 